In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression


In [2]:
class Logistic_regression:

    def __init__(self, X, Y):
        self.X, self.Y, self.beta = X, Y, np.zeros((X.shape[1], 1)).flatten()

    def log_loss(self):
        return np.dot(self.Y.T, np.log(self.sigmoid(self.X))) + np.dot((1 - self.Y).T, np.log(1 - self.sigmoid(self.X)))

    def cost_function(self):
        return -np.sum(self.log_loss()) + self.epsilon * np.sum(self.update_weights() ** 2)

    def update_weights(self):
        dev_grad = np.dot(self.X.T, (self.predict(self.X) - self.Y)) / self.Y.shape[0]
        self.beta = self.beta - self.step_size * (dev_grad - (self.epsilon * self.beta) / self.Y.shape[0])
        return self.beta

    def fit(self, max_steps=5000, step_size=0.000000404, epsilon=0.00000001):
        self.bias = 0
        self.step_size = step_size
        self.epsilon = epsilon
        for i in range(max_steps):
            self.beta = self.update_weights()
            self.bias = self.bias - step_size * (self.sigmoid(self.X) - self.Y).sum() / self.Y.shape[0]
        return self

    def predict(self, X):
        self.h = self.sigmoid(X)
        predict = np.where(self.h >= 0.5, 1, 0)
        return predict

    def sigmoid(self, X):
        return 1 / (1 + np.exp(-np.dot(X, self.beta) + self.bias))


In [3]:
default = pd.read_csv('classification.csv')
default.ed = default.ed.astype('category').cat.codes

In [4]:
X, y = default[default.columns.difference(['default'])], default.default
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)


In [6]:
log_reg = Logistic_regression(X_train, y_train)
log_reg.fit(max_steps=10000, step_size=0.0001, epsilon=1e-14)
train_pred = log_reg.predict(X_train)

In [7]:
# Confusion matrix from our implementation
cm = confusion_matrix(train_pred, y_train)
cm

array([[330,  64],
       [ 86,  80]])

In [8]:
# Confusion matrix from sklearn Logistic Regression model
classifier = LogisticRegression(random_state=0, max_iter=10000, solver='lbfgs', multi_class='auto')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
cm_1 = confusion_matrix(y_pred, y_train)
cm_1

array([[383,  70],
       [ 33,  74]])